**_Imported neccessary libraries for the project_**

In [1]:
import pandas as pd
import requests
import urllib.request
import bs4
import re

**_Used Requests to read URL and used BeautifulSoup to store webpage in XML format_**

In [2]:
data = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
data1 = bs4.BeautifulSoup(data, 'lxml')


**_In following two cells I found table data and store in variable 'row'_**

In [3]:
table = data1.find('table')


In [4]:
table_rows = data1.find_all('tr')

for tr in table_rows:
    td = data1.find_all('td')
    row = [i.text for i in td]


**_In this cell used Pandas to put table data into DataFrame_**

In [5]:
df = pd.DataFrame(row)
df.head()

,0
0,\nM1ANot assigned\n\n
1,\nM2ANot assigned\n\n
2,\nM3ANorth York(Parkwoods)\n\n
3,\nM4ANorth York(Victoria Village)\n\n
4,\nM5ADowntown Toronto(Regent Park / Harbourfro...


**_Renamed column to 'data'_**

In [6]:
df.rename(columns={0:'data'},inplace=True)
df.head()

,data
0,\nM1ANot assigned\n\n
1,\nM2ANot assigned\n\n
2,\nM3ANorth York(Parkwoods)\n\n
3,\nM4ANorth York(Victoria Village)\n\n
4,\nM5ADowntown Toronto(Regent Park / Harbourfro...


**_Eliminated '\n' characters_**

In [7]:
df['data'] = df['data'].map(lambda x: x.lstrip('\n').rstrip('\n\n'))
df.head()

,data
0,M1ANot assigned
1,M2ANot assigned
2,M3ANorth York(Parkwoods)
3,M4ANorth York(Victoria Village)
4,M5ADowntown Toronto(Regent Park / Harbourfront)


**_Created new column 'PostalCode', extracted postal code and moved it to 'PostalCode' column_**

In [8]:
df['PostalCode'] = df.data.str.extract('([M\d\D]\w{0,2})', expand=True)
df.data = df.data.str.replace('(M\d\D)', '')
df.head()

,data,PostalCode
0,Not assigned,M1A
1,Not assigned,M2A
2,North York(Parkwoods),M3A
3,North York(Victoria Village),M4A
4,Downtown Toronto(Regent Park / Harbourfront),M5A


**_Deleted 'Not assigned' rows'; deleted 'NaN' values in 'PostalCode' column; index reset_**

In [9]:
df1 = df[df['data'] != 'Not assigned']
df2 = df1[df1.PostalCode != 'NaN']
df2.reset_index(drop=True, inplace=True)
df2.head()

,data,PostalCode
0,North York(Parkwoods),M3A
1,North York(Victoria Village),M4A
2,Downtown Toronto(Regent Park / Harbourfront),M5A
3,North York(Lawrence Manor / Lawrence Heights),M6A
4,Queen's Park / Ontario Provincial Government,M7A


**_Extracted Borough information and storred in new column 'Borough'; \
Extracted Neighborhood information and storred in new column 'Neighborhood'; \
Replaced '\' sybol with commas;\
Dropped column 'data' and bottom empty rows_**

In [10]:
df2['Borough'] = df2.data.str.extract('([A-Za-z\s\') ]+)')
df2['Neighborhood'] = df2.data.str.extract('(\([A-Za-z\s\/\.\-\') ]+)',expand=False)
df2['Neighborhood'] = df2.Neighborhood.str.extract('([A-Za-z\s\/\.\-\' ]+)',expand=False)
df2['Neighborhood'] = df2['Neighborhood'].str.replace('(\ /)', ',')
df2.drop(columns=['data'], inplace=True)
df2 = df2[:-35]
df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,NaN


**_Replaced 'NaN' values in 'Neighborhood' column with data from 'Borough' column_**

In [11]:
df2['Neighborhood'] = df2['Neighborhood'].fillna(df2['Borough'])

In [513]:
df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [12]:
df2.shape

(103, 3)